
# 🇰🇷 RAG Quick Lab (40‑min) — Pinecone + KorQuAD 2.0 (Beginner)

이 노트북은 **한국어 RAG** 실습을 위해 준비되었습니다. (대상: RAG 처음인 대학원생)
- **임베딩**: `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` (384-dim, 빠르고 가벼움)
- **데이터셋**: KorQuAD 2.0 (일부 샘플) — 표·리스트가 포함된 한국어 위키 기반 MRC 데이터
- **Vector DB**: Pinecone (Serverless Index)

> 실행 전 준비물
> 1. Python ≥ 3.9 (권장 3.10+)
> 2. (선택) 가상환경 생성 후 활성화
> 3. **Pinecone** API Key 발급 → 환경변수 `PINECONE_API_KEY` 지정

참고:
- KorQuAD 2.0: https://korquad.github.io/
- Pinecone Quickstart: https://docs.pinecone.io/guides/indexes/create-an-index
- RAG 평가(RAGAS): https://docs.ragas.io/  (이번 랩에서는 간단 평가만)


## 0. Install packages

In [22]:

# (Colab or local) — 인터넷 연결 필요
!pip -q install "pinecone-client>=5,<6" "sentence-transformers>=2.4,<3" "datasets>=2.19,<3" "tqdm>=4.66,<5" "python-dotenv>=1.0,<2"
# RAG 평가까지 하려면 (선택)
!pip -q install "ragas>=0.1.14" "pandas>=2.1,<3"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas 0.3.5 requires datasets>=4.0.0, but you have datasets 2.21.0 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 1. 환경 변수 / 라이브러리 로드

In [23]:

import os, json, uuid, time, math, random, re
from dataclasses import dataclass
from typing import List, Dict, Any

from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# .env에 PINECONE_API_KEY=... 넣어두면 자동 로드
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not PINECONE_API_KEY:
    raise RuntimeError("환경변수 PINECONE_API_KEY 가 없습니다. Pinecone 콘솔에서 발급 후 설정하세요. (export PINECONE_API_KEY=...)")

print("✅ 환경 변수 로드 완료")


✅ 환경 변수 로드 완료


## 2. Pinecone 초기화 및 인덱스 생성 (Serverless)

In [27]:

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

# 장소는 문서 기준 예시입니다. (요금/리전은 계정 콘솔에서 확인)
CLOUD = "aws"          # 또는 "gcp","azure" (계정/요금제 확인)
REGION = "us-east-1"   # 예: "us-east-1", "us-west-2", "eu-west-1", "us-east-1-aws" 등 콘솔 권장 리전
INDEX_NAME = "rag-korquad-demo"

EMBED_DIM = 384  # MiniLM-L12-v2 임베딩 차원
METRIC = "cosine"

# 인덱스가 없으면 생성
if INDEX_NAME not in [idx["name"] for idx in pc.list_indexes()]:
    print(f"⏳ Creating index: {INDEX_NAME}")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBED_DIM,
        metric=METRIC,
        spec=ServerlessSpec(cloud=CLOUD, region=REGION),
        # Serverless는 pod 타입 미지정
    )
    # 인덱스 준비 대기 (간단 대기)
    time.sleep(10)
else:
    print(f"ℹ️ Index already exists: {INDEX_NAME}")

index = pc.Index(INDEX_NAME)
print(index.describe_index_stats())


⏳ Creating index: rag-korquad-demo
{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


## 3. KorQuAD 2.0 샘플 로드 & 청크 생성


- 본 실습은 **Hugging Face Datasets**의 커뮤니티 업로드된 *KorQuAD 2.0* 미러를 사용합니다.
- 네트워크 환경에 따라 로딩이 지연될 수 있습니다.
- 표/리스트가 포함된 긴 문서 특성상, **문단 단위 청크 + 살짝 겹침** 전략을 사용합니다.


In [25]:
import re
from datasets import load_dataset

DATASET_REPO = "leeseeun/KorQuAD_2.0"
raw = load_dataset(DATASET_REPO, split="train")

Q_PAT = re.compile(r"^[#\s]*질문\s*:\s*(.+?)\s*$", re.IGNORECASE | re.MULTILINE)
A_PAT = re.compile(r"^[#\s]*답변\s*:\s*(.+?)\s*$", re.IGNORECASE | re.MULTILINE)

def _extract_qna_from_text(text: str):
    q = a = ""
    if not text: 
        return q, a
    mq = Q_PAT.search(text)
    ma = A_PAT.search(text)
    if mq: q = mq.group(1).strip()
    if ma: a = ma.group(1).strip()
    return q, a

def normalize_sample(x):
    # 1) 기본 키
    q = x.get("question") or x.get("Question") or x.get("questions") or ""
    ctx = x.get("context")  or x.get("Context")  or x.get("article")   or ""
    ans = ""
    txt = x.get("text") or ""

    # 2) KorQuAD answers dict
    answers = x.get("answers")
    if isinstance(answers, dict):
        ans_list = answers.get("text", [])
        if isinstance(ans_list, list) and ans_list:
            ans = (ans_list[0] or "").strip()

    # 3) ★ 컨텍스트 폴백: context가 비어 있으면 text에서 파싱 (질문 유무와 무관)
    if not ctx and txt:
        q2, a2 = _extract_qna_from_text(txt)
        # 질문/정답이 비어있다면 보조로 채워줌
        if not q and q2: 
            q = q2
        if not ans and a2:
            ans = a2

        # RAG용 컨텍스트: Q/A 라인을 제거한 나머지 전체
        ctx_candidate = re.sub(r"^[#\s]*질문\s*:.+$", "", txt, flags=re.MULTILINE)
        ctx_candidate = re.sub(r"^[#\s]*답변\s*:.+$", "", ctx_candidate, flags=re.MULTILINE).strip()

        # 만약 제거하고 나니 진짜로 남는 게 없다면, 최후의 수단으로 txt 전체를 사용
        ctx = ctx_candidate if ctx_candidate else txt

    # 방어적 정리
    q   = str(q or "").strip()
    ctx = str(ctx or "").strip()
    ans = str(ans or "").strip()
    return {"question": q, "context": ctx, "answer": ans, "raw_text": txt}

dataset = raw.map(normalize_sample)
dataset = dataset.shuffle(seed=42).select(range(min(300, len(dataset))))
print(dataset[0])


{'question': '유럽찌르레기의 울음 소리와 비슷한 조류는 무엇인가?', 'answer': '수탉', 'text': '## 질문: 유럽찌르레기의 울음 소리와 비슷한 조류는 무엇인가?\n## 답변: 수탉\n\n', 'context': '## 질문: 유럽찌르레기의 울음 소리와 비슷한 조류는 무엇인가?\n## 답변: 수탉', 'raw_text': '## 질문: 유럽찌르레기의 울음 소리와 비슷한 조류는 무엇인가?\n## 답변: 수탉\n\n'}


### 3‑1. 간단 청크 함수 (문단 기반 + 겹침)

In [26]:
import re

def paragraph_chunk(text: str, max_chars=800, overlap_chars=120):
    if not text:
        return []
    paras = [p.strip() for p in re.split(r'(?:\r?\n){2,}', text) if p.strip()]
    if not paras:
        paras = [text.strip()]
    chunks = []
    for p in paras:
        if len(p) <= max_chars:
            chunks.append(p)
        else:
            start = 0
            while start < len(p):
                end = min(start + max_chars, len(p))
                chunks.append(p[start:end])
                if end >= len(p):
                    break
                start = max(0, end - overlap_chars)
    return chunks

corpus = []
skipped = 0
for i, row in enumerate(dataset):
    ctx = row["context"]
    if not ctx:
        skipped += 1
        continue
    for j, ch in enumerate(paragraph_chunk(ctx)):
        corpus.append({
            "id": f"doc-{i}-chunk-{j}",
            "text": ch,
            "meta": {
                "source": "KorQuAD2.0-mirror",
                "doc_id": f"doc-{i}",
                "chunk_id": j,
                "language": "ko",
                "has_true_context": bool(row["raw_text"] and row["context"] != "")
            }
        })

print("len(corpus) =", len(corpus), "| skipped(no context) =", skipped)
if corpus:
    print("preview:", corpus[0]["id"], corpus[0]["text"][:120])


len(corpus) = 301 | skipped(no context) = 0
preview: doc-0-chunk-0 ## 질문: 유럽찌르레기의 울음 소리와 비슷한 조류는 무엇인가?
## 답변: 수탉


## 4. 임베딩 생성 & Pinecone 업서트

In [28]:
from tqdm import tqdm
import math
import itertools
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def batched(iterable, n=64):
    it = iter(iterable)
    while True:
        batch = list(itertools.islice(it, n))
        if not batch:
            break
        yield batch

if not corpus:
    raise RuntimeError("corpus가 비어 있습니다. 앞 단계(정규화/청크)에서 ctx 폴백이 제대로 되었는지 확인하세요.")

for batch in tqdm(batched(corpus, 64), total=math.ceil(len(corpus)/64)):
    texts = [b["text"] for b in batch]
    vecs = model.encode(texts, normalize_embeddings=True, show_progress_bar=False)
    items = []
    for b, v in zip(batch, vecs):
        items.append({
            "id": b["id"],
            "values": v.tolist(),
            "metadata": {**b["meta"], "preview": b["text"][:200]}
        })
    # ★ v5: vectors= 로 전달
    index.upsert(vectors=items)  # (선택) namespace="default"
print("✅ Upsert 완료")


100%|██████████| 5/5 [00:04<00:00,  1.21it/s]

✅ Upsert 완료


## 5. 검색 테스트 — 한국어 질문으로 Top‑k 보기

In [29]:

TEST_QUERIES = [
    "주민등록표 초본 발급 수수료는 얼마인가?",
    "위성항법장치(GPS)의 원리는 무엇인가?",
    "대한민국 국회의 역할은 무엇인가?",
]

def search(query: str, top_k=5):
    qv = model.encode([query], normalize_embeddings=True)[0].tolist()
    res = index.query(vector=qv, top_k=top_k, include_metadata=True)
    return res

for q in TEST_QUERIES:
    print("\n### Q:", q)
    res = search(q, top_k=5)
    for i, match in enumerate(res["matches"]):
        score = match["score"]
        meta = match.get("metadata", {})
        preview = meta.get("preview", "")
        print(f"{i+1:>2}. score={score:.3f} | id={match['id']} | {preview[:90]}...")



### Q: 주민등록표 초본 발급 수수료는 얼마인가?
 1. score=0.465 | id=doc-189-chunk-0 | ## 질문: 2010년에 남태령역에서 승차한 하루 평균 인원은 몇 명일까?
## 답변: 1,316...
 2. score=0.455 | id=doc-287-chunk-0 | ## 질문: 2005년까지만 해도 이미 얼마만큼의 자금이 들었습니까?
## 답변: 130여억원...
 3. score=0.431 | id=doc-261-chunk-0 | ## 질문: 2004년부터 2009년까지 고료카쿠 역의 연도별 화물량은 어느 정도였을까?
## 답변:  연도 보냄 받음 합계 비고 2004년 199,000 174...
 4. score=0.416 | id=doc-60-chunk-0 | ## 질문: 이세이시바시역에서 승차하는 하루 평균 인원수가 73명인 해는?
## 답변: 2000년...
 5. score=0.402 | id=doc-41-chunk-0 | ## 질문: 로고 금액이 얼마야?
## 답변: 250만 유로...

### Q: 위성항법장치(GPS)의 원리는 무엇인가?
 1. score=0.386 | id=doc-242-chunk-0 | ## 질문: 케플러 관측선을 통해 알게 된 사실들에는 어떠한 것들이 있습니까?
## 답변: 2010년 6월 케플러 관측선과 통신연결이 시작된 지 43일이 지난 시...
 2. score=0.370 | id=doc-104-chunk-0 | ## 질문: 벤구리온 국제공항을 운영하는 곳은 어디야?
## 답변: 이스라엘 공항 공사...
 3. score=0.361 | id=doc-153-chunk-0 | ## 질문: 무기중에서 고출력 라스건의 특징에는 어떤것들이 있을까?
## 답변: 강력한 대구경 탄환을 사용한다. 또한 총검을 붙여놨기 때문에 근접전에서 상당한 위...
 4. score=0.340 | id=doc-193-chunk-0 | ## 질문: 지하철 2호선 한양대역과 교내 본관 앞 연결통로의 이름은 무엇인가? 


### 5‑1. (옵션) 간이 생성 템플릿 — LLM 없이 스니펫 기반

In [30]:

def answer_with_snippets(query: str, top_k=5):
    res = search(query, top_k=top_k)
    snippets = [m["metadata"]["preview"] for m in res["matches"]]
    ans = f"질문: {query}\n\n아래 근거 스니펫을 참고하세요:\n" + "\n---\n".join(snippets)
    return ans

print(answer_with_snippets("주민등록표 초본 발급 수수료는 얼마인가?"))


질문: 주민등록표 초본 발급 수수료는 얼마인가?

아래 근거 스니펫을 참고하세요:
## 질문: 2010년에 남태령역에서 승차한 하루 평균 인원은 몇 명일까?
## 답변: 1,316
---
## 질문: 2005년까지만 해도 이미 얼마만큼의 자금이 들었습니까?
## 답변: 130여억원
---
## 질문: 2004년부터 2009년까지 고료카쿠 역의 연도별 화물량은 어느 정도였을까?
## 답변:  연도 보냄 받음 합계 비고 2004년 199,000 174,000 373,000 2005년 199,000 159,000 358,000 2006년 205,000 156,000 361,000 2007년 187,000 148,000 335,000 2008년 1
---
## 질문: 이세이시바시역에서 승차하는 하루 평균 인원수가 73명인 해는?
## 답변: 2000년
---
## 질문: 로고 금액이 얼마야?
## 답변: 250만 유로


## 6. 간단한 Retrieval 품질 측정 (Recall@k)


- **아이디어**: KorQuAD 정답 문자열이 포함된 청크가 Top‑k에 들어오면 **정답을 찾은 것**으로 간주
- 주의: 실제 평가는 토큰화/정규화, 부분일치, 라벨링 오류 등을 고려해야 합니다. (여기선 간단 버전)


In [31]:

def recall_at_k(samples, k=5, n_eval=50):
    # 무작위 질문 n_eval개 샘플링
    samp = random.sample(list(range(len(samples))), min(n_eval, len(samples)))
    hits = 0
    for i in tqdm(samp):
        row = samples[i]
        q, ans = row["question"], (row["answer"] or "").strip()
        if not ans: 
            continue
        res = search(q, top_k=k)
        top_texts = [m["metadata"].get("preview","") for m in res["matches"]]
        # 매우 단순한 포함 검사
        if any(ans in t for t in top_texts):
            hits += 1
    return hits / max(1, len(samp))

r5 = recall_at_k(dataset, k=5, n_eval=50)
print(f"Recall@5 (단순 포함 검사 기준) = {r5:.2%}")


100%|██████████| 50/50 [00:13<00:00,  3.64it/s]

Recall@5 (단순 포함 검사 기준) = 82.00%


## 7. (선택) 정리 — 인덱스 삭제

In [ ]:

# 실습 리소스 정리를 원할 때만 사용하세요.
# pc.delete_index(INDEX_NAME)
# print("🧹 Index deleted")



---

## Appendix — Notes & Links

- **KorQuAD 2.0**  
  - 홈페이지: https://korquad.github.io/
  - (커뮤니티 미러) Hugging Face: 예) https://huggingface.co/datasets/leeseeun/KorQuAD_2.0
- **Pinecone**  
  - Create Serverless Index: https://docs.pinecone.io/guides/indexes/create-an-index
- **평가**  
  - RAGAS: https://docs.ragas.io/ (이번 랩은 단순 Recall@k만 예시)

권장 과제 아이디어:
1) **하이브리드 검색**: BM25(예: `rank-bm25`) 결과와 Cosine을 RRF로 융합해 Top‑k 비교
2) **리랭크**: 상위 N 스니펫을 Cross-Encoder로 재점수화 (Cohere Rerank, Sentence-Transformers cross-encoder 등)
3) **메타 필터**: `language='ko'`, `source='KorQuAD2.0'` 등 필드로 필터링 → 품질/속도 비교
4) **청크 전략 비교**: 청크 크기/겹침을 달리하여 Recall@k 변화를 측정
